In [3]:
import sqlite3
import pandas as pd
import numpy as np
import re

In [4]:
db = sqlite3.connect("crawler_cache_clean.db")

In [5]:
cur = db.cursor()

In [6]:
cur.execute("select * from crawler_cache;")

In [7]:
results = cur.fetchall()

In [8]:
print len(results)

1256


In [9]:
def get_dir_url(num):
    return str(ur'https://www.kinopoisk.ru/name/') + re.findall(ur'(?<=\<td itemprop="director"><a href="/name/)(.*?)(?=\/")', results[num][1])[0]

In [10]:
get_dir_url(1)

u'https://www.kinopoisk.ru/name/242662'

In [11]:
import requests
from lxml import html

page = requests.get(get_dir_url(0))
tree = html.fromstring(page.content)
dir_rank = root.xpath('//*[@id="avg_rating"]')

NameError: name 'root' is not defined

In [ ]:
#print page.text
with open('tst.txt', "w") as f:
    f.write(page.content)
    f.close()

In [ ]:
!pip install dryscrape

In [ ]:
import dryscrape
from bs4 import BeautifulSoup
session = dryscrape.Session()
session.visit(get_dir_url(0))
response = session.body()
soup = BeautifulSoup(response)
soup.find(id="avg_rating")

In [12]:
def get_info(num):
    genre = list()
    genre += re.findall(ur'(?<=\<a href="/lists/m_act%5Bgenre%5D/[0-9]/" data-popup-info="disabled">)(.*?)(?=\</a>)', results[num][1])
    genre += re.findall(ur'(?<=\<a href="/lists/m_act%5Bgenre%5D/[0-9][0-9]/" data-popup-info="disabled">)(.*?)(?=\</a>)', results[num][1])
    genre += re.findall(ur'(?<=\<a href="/lists/m_act%5Bgenre%5D/[0-9]/">)(.*?)(?=\</a>)', results[num][1])
    genre += re.findall(ur'(?<=\<a href="/lists/m_act%5Bgenre%5D/[0-9][0-9]/">)(.*?)(?=\</a>)', results[num][1])
    set_genre = set(genre[0:min(len(genre), 2)])
    
    string_ranking = re.findall(ur'(?<=\<span class="rating_ball">)(.*?)(?=\</span>)', results[num][1])
    if len(string_ranking) == 0:
        ranking = -1
    else:
        ranking = float(string_ranking[0])

    country = list()
    country += re.findall(ur'(?<=\<a href="/lists/m_act%5Bcountry%5D/[0-9]/" data-popup-info="disabled">)(.*?)(?=\</a>)', results[num][1])
    country += re.findall(ur'(?<=\<a href="/lists/m_act%5Bcountry%5D/[0-9][0-9]/" data-popup-info="disabled">)(.*?)(?=\</a>)', results[num][1])
    country += re.findall(ur'(?<=\<a href="/lists/m_act%5Bcountry%5D/[0-9][0-9]/" data-popup-info="disabled">)(.*?)(?=\</a>)', results[num][1])
    country += re.findall(ur'(?<=\<a href="/lists/m_act%5Bcountry%5D/[0-9]/">)(.*?)(?=\</a>)', results[num][1])
    country += re.findall(ur'(?<=\<a href="/lists/m_act%5Bcountry%5D/[0-9][0-9]/">)(.*?)(?=\</a>)', results[num][1])
    country += re.findall(ur'(?<=\<a href="/lists/m_act%5Bcountry%5D/[0-9][0-9]/">)(.*?)(?=\</a>)', results[num][1])
    country = filter(lambda item : len(item) > 0, country)
    set_country = set(country[0:min(len(country), 1)])
    
    return [ranking, set_genre, set_country]

In [13]:
cur_film = 9
print results[cur_film][0]
film_info = []
film_info = get_info(cur_film)

https://www.kinopoisk.ru/film/trockiy-2017-1008339/


In [14]:
for x in film_info[1]:
    print x
print film_info[0]
for x in film_info[2]:
    print x

6.761
Россия


In [15]:
countries = set()
genres = set()
bad_films = set()
no_country = set()
less_genre = set()

In [16]:
for cur_film in range(0, len(results)):
    film_info = []
    film_info = get_info(cur_film);
    if(film_info[0] == -1):
        bad_films.add(cur_film)
        continue
    if len(film_info[1]) != 2:
        less_genre.add(cur_film)
    if len(film_info[2]) != 1:
        no_country.add(cur_film)
    for x in film_info[1]:
        genres.add(x)
    for x in film_info[2]:
        countries.add(x)

In [17]:
print len(countries)
print len(genres)
print 'no rank:', bad_films
print '0 or 1 genre:', less_genre
print 'no country', no_country

25
22
no rank: set([128, 576, 642, 1253, 390, 8, 139, 1215, 143, 1040, 17, 147, 20, 534, 408, 32, 1254, 1041, 1062, 41, 44, 45, 304, 1255, 51, 1216, 565, 479, 458, 319, 64, 1248, 1090, 67, 1249, 74, 77, 336, 83, 980, 86, 87, 1252, 92, 222, 95, 96, 609, 1250, 1251, 228, 101, 627, 401, 104, 875, 1132, 1134, 499, 628, 937, 376, 1017, 895, 383])
0 or 1 genre: set([515, 9, 1036, 525, 1027, 533, 819, 28, 1054, 1055, 546, 547, 1068, 1117, 57, 1082, 1084, 66, 68, 75, 595, 599, 954, 94, 97, 611, 1129, 619, 112, 626, 21, 1159, 137, 1164, 141, 142, 1173, 664, 1177, 1184, 673, 1186, 166, 170, 684, 689, 180, 181, 1209, 186, 1214, 703, 1217, 707, 801, 716, 205, 718, 1232, 1236, 728, 804, 1245, 235, 238, 753, 755, 759, 760, 762, 252, 765, 258, 784, 790, 287, 289, 292, 811, 813, 307, 311, 829, 836, 842, 851, 346, 348, 351, 865, 868, 876, 365, 879, 370, 896, 389, 391, 1004, 911, 400, 402, 919, 922, 411, 931, 420, 942, 433, 439, 441, 442, 956, 961, 1099, 452, 456, 969, 461, 465, 978, 932, 987, 478, 1190

In [18]:
all_info = np.ndarray([len(results) - len(bad_films), len(countries) + len(genres)], dtype=float)
all_rank = np.ndarray([len(results) - len(bad_films), 1], dtype=float)

In [19]:
it = 0
info = []
list_genres = list(genres)
list_countries = list(countries)
for x in range(0, len(results)):
    if(it % 100 == 0):
        print it, x
    if x in bad_films:
        continue
    info = get_info(x)
    cur_genres = list(info[1])
    cur_country = list(info[2])
    for i in xrange(0, len(genres)):
        if((len(cur_genres) > 0 and cur_genres[0] == list_genres[i]) or (len(cur_genres) > 1 and cur_genres[1] == list_genres[i])):
            all_info[it][i] = 1
        else:
            all_info[it][i] = 0
            
    for i in xrange(0, len(countries)):
        if(cur_country[0] == list_countries[i]):
            all_info[it][len(genres) + i] = 1
        else:
            all_info[it][len(genres) + i] = 0
            
    all_rank[it] = info[0]
    
    it = it + 1

0 0
100 120
200 225
300 328
400 434
500 538
600 644
700 744
800 844
900 947
1000 1051
1100 1155


In [20]:
print all_rank

[[ 6.903]
 [ 7.771]
 [ 7.977]
 ..., 
 [ 6.364]
 [ 7.329]
 [ 5.469]]


In [21]:
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model

In [22]:
regr = linear_model.LinearRegression()

In [23]:
regr.fit(all_info, all_rank.reshape(1, -1)[0])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [24]:
pred_films = list()

In [25]:
pred_films.append(ur'https://www.kinopoisk.ru/film/tayna-koko-2017-679486/')
pred_films.append(ur'https://www.kinopoisk.ru/film/proshchay-kristofer-robin-2017-993579/')
pred_films.append(ur'https://www.kinopoisk.ru/film/chastnoe-pionerskoe-3-privet-vzroslaya-zhizn-2017-909664/')
pred_films.append(ur'https://www.kinopoisk.ru/film/obet-molchaniya-2017-1009473/')
pred_films.append(ur'https://www.kinopoisk.ru/film/detki-naprokat-2017-1047968/')
pred_films.append(ur'https://www.kinopoisk.ru/film/mister-felichita-2017-1026440/')
pred_films.append(ur'https://www.kinopoisk.ru/film/legenda-o-kolovrate-2017-844118/')
pred_films.append(ur'https://www.kinopoisk.ru/film/inostranets-2017-618287/')
pred_films.append(ur'https://www.kinopoisk.ru/film/maksimalnyy-udar-2017-917942/')
pred_films.append(ur'https://www.kinopoisk.ru/film/atlantida-2017-678639/')
pred_films.append(ur'https://www.kinopoisk.ru/film/konvert-2017-981348/')

In [26]:
pred_films_genres = list()

In [27]:
pred_films_genres.append([ur'мультфильм', ur'мюзикл'])
pred_films_genres.append([ur'семейный', ur'биография'])
pred_films_genres.append([ur'комедия', ur'приключения'])
pred_films_genres.append([ur'боевик', ur'драма'])
pred_films_genres.append([ur'комедия', ur'семейный'])
pred_films_genres.append([ur'комедия'])
pred_films_genres.append([ur'история', ur'боевик'])
pred_films_genres.append([ur'боевик', ur'триллер'])
pred_films_genres.append([ur'комедия', ur'боевик'])
pred_films_genres.append([ur'фантастика', ur'ужасы'])
pred_films_genres.append([ur'ужасы', ur'триллер'])

In [28]:
pred_films_contr = list()

In [29]:
pred_films_contr.append([ur'США'])
pred_films_contr.append([ur'Великобритания'])
pred_films_contr.append([ur'Россия'])
pred_films_contr.append([ur'США'])
pred_films_contr.append([ur'Россия'])
pred_films_contr.append([ur'Италия'])
pred_films_contr.append([ur'Россия'])
pred_films_contr.append([ur'Великобритания'])
pred_films_contr.append([ur'Россия'])
pred_films_contr.append([ur'Испания'])
pred_films_contr.append([ur'Россия'])

In [30]:
list_genres = list(genres)
list_countries = list(countries)
def to_vec(num):
    ans = np.ndarray([1, len(countries) + len(genres)], dtype=float)
    
    for i in xrange(0, len(genres)):
        if((len(pred_films_genres[num]) > 0 and pred_films_genres[num][0] == list_genres[i]) or (len(pred_films_genres[num]) > 1 and pred_films_genres[num][1] == list_genres[i])):
            ans[0][i] = 1
        else:
            ans[0][i] = 0
            
    for i in xrange(0, len(countries)):
        if(pred_films_contr[num][0] == list_countries[i]):
            ans[0][len(genres) + i] = 1
        else:
            ans[0][len(genres) + i] = 0
    return ans

In [31]:
for i in range(0, 11):
    print pred_films[i], regr.predict(to_vec(i).reshape(1, -1))[0]

https://www.kinopoisk.ru/film/tayna-koko-2017-679486/ 9.443359375
https://www.kinopoisk.ru/film/proshchay-kristofer-robin-2017-993579/ 7.693359375
https://www.kinopoisk.ru/film/chastnoe-pionerskoe-3-privet-vzroslaya-zhizn-2017-909664/ 6.3828125
https://www.kinopoisk.ru/film/obet-molchaniya-2017-1009473/ 7.4765625
https://www.kinopoisk.ru/film/detki-naprokat-2017-1047968/ 6.845703125
https://www.kinopoisk.ru/film/mister-felichita-2017-1026440/ 7.173828125
https://www.kinopoisk.ru/film/legenda-o-kolovrate-2017-844118/ 6.16015625
https://www.kinopoisk.ru/film/inostranets-2017-618287/ 6.97265625
https://www.kinopoisk.ru/film/maksimalnyy-udar-2017-917942/ 6.279296875
https://www.kinopoisk.ru/film/atlantida-2017-678639/ 6.7421875
https://www.kinopoisk.ru/film/konvert-2017-981348/ 6.044921875


In [32]:
for i in range(0, 11):
    print to_vec(i)

[[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]
[[ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]
[[ 0.  0.  0.  1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0

In [40]:
import csv

In [41]:
def make_res():
    with open('TS_Nakhodnov_task3.tsv', 'wb') as csvfile:
        spamwriter = csv.writer(csvfile, delimiter='\t')
        for i in range(0, len(pred_films)):
            spamwriter.writerow([pred_films[i], regr.predict(to_vec(i).reshape(1, -1))[0] * 0.93, 'Nakhodnov'])

In [42]:
make_res()